# Doc2VecでWikipedia文書を使用し、マイオペのデータで予測

巨大コーパスをもつWiki文書の一部を使用して、Wiki文書でボキャブラリを作成し、マイオペの学習セットで学習／予測させます。

今回は２つの手法を比較しました。

- PV-DBOW：マイオペの学習セットを混ぜて学習させることで、マイオペのデータでも予測ができることは確認できました。


- PV-DM：こちらは全般的に使えない印象です。

## (1) Wikipediaコンテンツファイルから全文書を抽出

<a href="31-Wikipedia-contents-csv.ipynb"><b>こちらの手順</b></a> にて、いったんローカルPCにCSVファイル化しておきます。



## (2) TaggedDocumentを生成

### (2-1) Wikipedia文書のTaggedDocumentを生成

Wikipedia文書のCSVファイルからデータフレームを読み込み、TaggedDocumentに変換します。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [3]:
from gensim.models.doc2vec import TaggedDocument
# from learning.core.nlang import Nlang

def get_tagged_document_list(wiki_ids, wiki_contents):
    '''
        タグとコーパスをTaggedDocumentに設定
    '''
    tagged_document_list = []

    for index, _ in enumerate(wiki_ids):
        tag = wiki_ids[index]
        sentences = wiki_contents[index]

        # 品詞は落とさない様にする
        separated_sentences = Nlang_naive.split(sentences)
        words = separated_sentences.split()
        tagged_document_list.append(TaggedDocument(words=words, tags=[tag]))
        
    return tagged_document_list

In [4]:
'''
    Wikipedia文書から生成されたCSVを読み込み、
    配列を生成する
'''
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'
csv_file_name = os.path.join(extracted_dir_path, 'Wikipedia-content.csv')
df = pd.read_csv(csv_file_name)

_wiki_ids = np.array(df['id'])
_wiki_titles = np.array(df['title'])
_wiki_contents = np.array(df['content'])

In [16]:
'''
    時間がかかるので一部のみ使用します（全量＝100万件）
'''
partial_cnt = 1000

wiki_ids = _wiki_ids[0:partial_cnt]
wiki_contents = _wiki_contents[0:partial_cnt]

In [17]:
wiki_tagged_document_list = get_tagged_document_list(wiki_ids, wiki_contents)

In [18]:
len(wiki_tagged_document_list)

1000

### (2-2) マイオペ文書のTaggedDocumentを生成

In [19]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 11 # benefitone.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [20]:
'''
    マイオペの学習セットを取得してコーパスを生成
    
    タグとして、learning_training_messages.id を付与
'''
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])
ids = np.array(learning_training_messages['id'])

myope_tagged_document_list = get_tagged_document_list(ids, questions)

2017/05/25 PM 01:50:42 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/25 PM 01:50:42 ['./fixtures/question_answers/toyotsu_human.csv']


In [21]:
len(myope_tagged_document_list)

7083

### (2-2) Wikipedia文書とマイオペ文書の、両方のTaggedDocumentを合体させる

In [22]:
all_tagged_document_list = []
all_tagged_document_list.extend(wiki_tagged_document_list)
all_tagged_document_list.extend(myope_tagged_document_list)

In [23]:
len(all_tagged_document_list)

8083

## (3) 学習実行

### (3-1) TaggedDocumentをインプットとして学習

In [24]:
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

def build_vocabulary(tagged_document_list, dm=0, size=100):
    '''
        tagged_document_listは、
        Wikipedia文書のタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
    '''
    model = Doc2Vec(dm=dm, size=size, min_count=1, iter=size*10)
    model.build_vocab(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('build_vocabulary: model saved temporary')

    return model

def train(model, tagged_document_list, dm=0):
    '''
        tagged_documentが収容されたリストを引数にして、
        学習を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        （今回は比較のため、両方で試す）
    '''
    ret = model.train(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('train: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [25]:
'''
    tagged_document_listを使用し、学習実行（PV-DBOW）
'''
model_dbow = build_vocabulary(all_tagged_document_list, dm=0, size=200)

build_vocabulary: model saved temporary


In [26]:
train(model_dbow, all_tagged_document_list, dm=0) 

train: document vector size=8083, return=2472651108


2472651108

In [34]:
'''
    tagged_document_listを使用し、学習実行（PV-DM）
'''
model_dm = build_vocabulary(all_tagged_document_list, dm=1, size=200)

build_vocabulary: model saved temporary


In [35]:
train(model_dm, all_tagged_document_list, dm=1) 

train: document vector size=8083, return=2472651949


2472651949

## (3) 予測実行

作成されたモデルを使用して予測を実行する関数になります。

In [39]:
from gensim import models

def predict(word, dm=0):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(dm))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

def get_title_from_tag(tag):
    '''
        tag に対応するWikipediaのタイトルを取得。

        ただし、合致したものがなければ、
        マイオペの id と判断し、
        id に対応する answer_id を取得する。
    '''
    for index, _ in enumerate(wiki_ids):
        if wiki_ids[index] == tag:
            return _wiki_titles[index]

    for index, _ in enumerate(ids):
        if ids[index] == tag:
            return answer_ids[index]
        
    return None

def print_predict_result(corpus, similarities):
    print(corpus)
    print("id, answer_id, similarity")

    for tag, similarity in similarities:
        print(tag, get_title_from_tag(tag), similarity)

### (3-1) 適当な質問文で予測（PV-DBOW)

#### 正解があるものは高めの類似度を示します

In [28]:
'''
    契約書を見たいのですが（正解＝4683）
'''
corpus, ret = predict('契約書を見たいのですが', dm=0)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が']
id, answer_id, similarity
1898658 4683 0.5975109934806824
1895471 4683 0.5846216082572937
1898648 4683 0.5720071196556091
1895461 4683 0.5661964416503906
1898654 4683 0.5602788925170898
1896954 4678 0.5566464066505432
1895466 4683 0.5562644004821777
1898653 4683 0.5544137954711914
1895469 4683 0.5539995431900024
1895467 4683 0.5511842370033264


In [29]:
'''
    EXカードを貸してください（正解＝4678）
'''
corpus, ret = predict('EXカードを貸してください', dm=0)
print_predict_result(corpus, ret)

['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる']
id, answer_id, similarity
1899813 4678 0.7045131921768188
1896626 4678 0.6912281513214111
1901396 4678 0.6634103059768677
1898215 4678 0.6614203453063965
1901402 4678 0.6600054502487183
1898209 4678 0.6520925760269165
1899807 4678 0.6442662477493286
1899261 4740 0.631352961063385
1899217 4678 0.6307018399238586
1901397 4678 0.6267096996307373


#### 正解があるものについて、言い回しを変えてみます。

いずれも、想定される結果からは、かけ離れたものになってしまいます。

In [48]:
'''
    正解の否定（教師データにない）---> 否定が効かない
    
    （「契約書を見たいのですが」の正解＝4683）
'''
corpus, ret = predict('契約書を見せたくないのですが', dm=0)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見せる', 'たい', 'ない', 'の', 'です', 'が']
id, answer_id, similarity
1898648 4683 0.5675591230392456
1895461 4683 0.5639031529426575
1895466 4683 0.563579261302948
1898653 4683 0.5598781108856201
1895469 4683 0.5595439672470093
1898656 4683 0.5562201142311096
1898654 4683 0.5555241107940674
1898813 4711 0.5551542043685913
1895467 4683 0.5543932914733887
1901043 4678 0.5459620952606201


In [53]:
'''
    正解の別な言い回し（教師データにない）---> 別の答えが返ってきてしまう

    （「契約書を見たいのですが」の正解＝4683）
'''
corpus, ret = predict('契約を結んだ時に作成される文書を確認したい', dm=0)
print_predict_result(corpus, ret)

['契約', 'を', '結ぶ', 'だ', '時', 'に', '作成', 'する', 'れる', '文書', 'を', '確認', 'する', 'たい']
id, answer_id, similarity
1895356 4797 0.5864300727844238
1902121 4797 0.5848488807678223
1900376 4678 0.582711935043335
1897185 4678 0.5824649333953857
1900372 4678 0.5812031030654907
1897189 4678 0.580223798751831
1898656 4683 0.5747554302215576
1895466 4683 0.5707365274429321
1895469 4683 0.570685088634491
1898653 4683 0.5705815553665161


#### 正解がないものは低めの類似度を示します

In [30]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
corpus, ret = predict('おいしいラーメンが食べたいです', dm=0)
print_predict_result(corpus, ret)

['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です']
id, answer_id, similarity
1900597 4678 0.5136572122573853
1896814 4678 0.5046058893203735
1900001 4678 0.5039975047111511
1897410 4678 0.5036501884460449
1900034 4678 0.4947577118873596
1896954 4678 0.4943247139453888
1896990 4678 0.49368739128112793
1896630 4678 0.49354636669158936
1896847 4678 0.4925711154937744
1900177 4678 0.492549866437912


In [31]:
'''
    無関係な質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=0)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
id, answer_id, similarity
1896638 4678 0.4809154272079468
1899995 4678 0.4796925187110901
1899825 4678 0.4765402376651764
1896808 4678 0.47560590505599976
1900298 4783 0.4701211452484131
1901188 4678 0.46987462043762207
1897105 4783 0.4698534309864044
1900949 4678 0.4662317931652069
1901120 4678 0.4659349322319031
1897111 4783 0.4648154079914093


#### マイオペとは無関係な、Wikipedia文書に関するデータの予測でも、マイオペの文書が帰ってきてしまいます

ただし類似度は低めです

In [32]:
'''
    Wikiタイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=0)
print_predict_result(corpus, ret)

['風邪']
id, answer_id, similarity
1897733 4678 0.4665856957435608
1898452 4833 0.4632178544998169
1900920 4678 0.45248931646347046
1898458 4833 0.4482731223106384
1899816 4678 0.4474280774593353
1901639 4833 0.44547876715660095
1899739 4772 0.4439881443977356
1898075 4678 0.443499892950058
1901262 4678 0.4429112672805786
1900674 4678 0.43948814272880554


In [33]:
'''
    Wiki本文に近い文言で、質問をしてみます。
'''
wiki_like_sentence = '''漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という。
'''
corpus, ret = predict(wiki_like_sentence, dm=0)
print_predict_result(corpus, ret)

['漁業', 'に', 'は', 'さまざま', 'だ', '人々', 'が', '関わる', 'て', 'いる', 'が', '漁撈', '活動', 'に', '専業', 'として', '携わる', '者', 'の', 'こと', 'は', '漁師', 'と', 'いう']
id, answer_id, similarity
1899750 4772 0.43229085206985474
1900340 4678 0.4295662045478821
1900297 4783 0.4275372624397278
1900073 4678 0.4240705966949463
1900288 4783 0.42388951778411865
1896563 4772 0.4207908511161804
1901034 4678 0.4207456111907959
1899983 4678 0.4195511043071747
1899763 4772 0.41805917024612427
1896886 4678 0.41684645414352417


### (3-2) 適当な質問文で予測（PV-DM)

In [40]:
'''
    契約書を見たいのですが（正解＝4683）
'''
corpus, ret = predict('契約書を見たいのですが', dm=1)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が']
id, answer_id, similarity
1901609 4808 0.5073404312133789
1901605 4808 0.5008575916290283
1898422 4808 0.4915803074836731
1898658 4683 0.48995378613471985
1901615 4808 0.48584046959877014
1898416 4808 0.4836342930793762
1901603 4808 0.4823327958583832
1899889 4678 0.4742242097854614
1898428 4808 0.47137218713760376
1898418 4808 0.4703996777534485


In [41]:
'''
    EXカードを貸してください（正解＝4678）
'''
corpus, ret = predict('EXカードを貸してください', dm=1)
print_predict_result(corpus, ret)

['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる']
id, answer_id, similarity
1898961 4730 0.5005889534950256
1901294 4678 0.500424861907959
1901360 4678 0.49449506402015686
1895774 4730 0.49340152740478516
1898107 4678 0.4931187033653259
1898949 4729 0.49211177229881287
1898966 4731 0.49201926589012146
1898945 4729 0.49172648787498474
1900132 4678 0.49169832468032837
1898955 4729 0.4909236431121826


正解がない場合はWikiの文書から類似しているものを引っ張り出してくる様です。

（Wikiからは、doc id="1316" の「ワンダフルライフ (映画)」というタイトルがついた文書を引っ張ってきています）

In [42]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
corpus, ret = predict('おいしいラーメンが食べたいです', dm=1)
print_predict_result(corpus, ret)

['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です']
id, answer_id, similarity
1895270 4678 0.19972434639930725
1895419 4835 0.18640415370464325
1900841 4678 0.18618609011173248
1316 ワンダフルライフ (映画) 0.1834898293018341
1895388 4835 0.18271973729133606
1895642 4712 0.1812029927968979
1898110 4678 0.17716899514198303
1416 直木三十五賞 0.17708000540733337
1898829 4712 0.17440631985664368
1898136 4678 0.17416459321975708


In [43]:
'''
    無関係な質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=1)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
id, answer_id, similarity
1897454 4678 0.1340162307024002
1897445 4678 0.13301610946655273
1900641 4678 0.1324583888053894
1900632 4678 0.13190717995166779
1897439 4678 0.13065944612026215
1900626 4678 0.12922430038452148
1530 相撲 0.11540471017360687
1078 必ずお読み下さい。 0.07361582666635513
1314 富士通 0.062489062547683716
792 癇癪玉 0.05928400158882141


#### 風邪についての質問

Wikiの文書から、doc id="329" の「風邪」というタイトルがついた文書を引っ張ってきています。

ただし類似度が一番高いのはマイオペの id=1899583（イヤホンが壊れた）という質問文でした。

In [44]:
'''
    Wikiタイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=1)
print_predict_result(corpus, ret)

['風邪']
id, answer_id, similarity
1899583 4761 0.41115620732307434
329 風邪 0.40796104073524475
1901817 4686 0.4034441113471985
1897221 4678 0.4000726342201233
1900301 4783 0.39073067903518677
1901716 4797 0.38969072699546814
1898529 4797 0.389523446559906
1898966 4731 0.3883591294288635
1901979 4678 0.3881809711456299
1895516 4692 0.38608691096305847


#### Wiki 文書の一部から抜粋した文章で予測しましたが、なぜかマイオペの文書が帰ってしまいます。

類似度が一番高いのはマイオペの id=1900560（利用者（発送者）を確認できるのでしょうか？）という質問文でした。

In [45]:
'''
    Wiki本文に近い文言で、質問をしてみます。
'''
wiki_like_sentence = '''漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という。
'''
corpus, ret = predict(wiki_like_sentence, dm=1)
print_predict_result(corpus, ret)

['漁業', 'に', 'は', 'さまざま', 'だ', '人々', 'が', '関わる', 'て', 'いる', 'が', '漁撈', '活動', 'に', '専業', 'として', '携わる', '者', 'の', 'こと', 'は', '漁師', 'と', 'いう']
id, answer_id, similarity
1900560 4786 0.4250141978263855
1900559 4786 0.41877880692481995
1897372 4786 0.41853368282318115
1900564 4786 0.40057089924812317
1899109 4678 0.3985738754272461
1897373 4786 0.3980163335800171
1899106 4678 0.39679884910583496
1900563 4786 0.3912838101387024
1899938 4678 0.389983594417572
1900292 4783 0.3876217007637024
